In [4]:
from PIL import Image

def get_points(picture_path):
    with Image.open(picture_path) as img:
        colors = img.getcolors(2)
        if (colors == None):
            raise Exception("Only monochrome images are supported")
        [(_, black), (_, white)] = colors
        return [(x, y) for x in range(img.width) for y in range(img.height) if img.getpixel((x,y)) == black]

points = get_points("sample.png")
    
plane = AffineSpace(QQ, 2, 'x')
flat_pictures_space = cartesian_product([plane for _ in points])
picture_point = flat_pictures_space(plane(p) for p in points)
# select an image of our picture in the "symmetric power":
# 1. Take a subring of functions symmetric in points in the coordinate ring of flat_pictures_space.

# 2. Make an affine variety with such a ring as a coordinate ring.
# 3. Make a morphism of varieties from an injective homomorphism.
# 4. Take an image of picture_point with this morphism.